In [9]:
def counts(gene, dna):
    for i in range(len(dna)):
        yield dna[i:i+len(gene)] == gene

def dna_health(health, genes, first, last, dna):
    return sum([
        sum(counts(genes[i], dna)) * health[i]
        for i in range(first, last+1)
    ])

In [31]:
class Node(object):
    def __init__(self, path, character, parent, suffix=None):
        self.path = path
        self.character = character
        self.parent = parent
        self.suffix = suffix
        self.health = None
    
    def __repr__(self):
        return '<Node character:{}, parent:{}, suffix:{} >'.format(
            self.character,
            '--' if not self.parent else '<Node path:'+self.parent.path+'>',
            '--' if not self.suffix else '<Node path:'+self.suffix.path+'>'
        )

In [55]:
class Node(object):
    def __init__(self, path, in_dict, parent):
        self.path = path
        self.children = {}
        self.parent = parent
        self.in_dict = in_dict
        self.suffix = None
        self.dict_suffix = None

    def __getitem__(self, letter):
        return self.children.get(letter)
    
    def __contains__(self, letter):
        return letter in self.children
    
    def __setitem__(self, letter, node):
        self.children[letter] = node
    
    def __repr__(self):
        return '{}\t\t{}\t\t{}\t\t{}\n'.format(
            "'"+self.path+"'",
            self.in_dict,
            '--' if not self.suffix else "'"+self.suffix.path+"'",
            '--' if not self.dict_suffix else "'"+self.dict_suffix.path+"'"
        ) + ''.join([
            child.__repr__() 
            for child in self.children.values()
        ])


class Trie(object):
    def __init__(self, dictionary, values):
        self.trie_root = Node('', False, None)

        for index, gene in enumerate(dictionary):
            curr_node = self.trie_root
            for i, letter in enumerate(gene):
                if letter not in curr_node:
                    in_dict = i == len(gene) - 1
                    curr_node[letter] = Node(gene[:i+1], in_dict, curr_node)
                else:
                    pass  # update health value
                curr_node = curr_node[letter]
        
        self._put_suffixes(self.trie_root, '')
        self._put_dict_suffixes(self.trie_root)

    def count(self, haystack):
        needles = 0
        node = self.trie_root
        for straw in haystack:
            node = self._next_good_node(node, straw)
            if node.in_dict:
                needles += 1
            needles += len(self._num_dict_suffixes(node))
        return needles
    
    def _num_dict_suffixes(self, node):
        suffixes = []
        suffix = node.dict_suffix
        while suffix:
            suffixes.append(suffix)
            suffix = suffix.dict_suffix
        return suffixes
    
    def _next_good_node(self, node, letter):
        potential = node[letter]
        base = node
        while not potential:
            potential = base.suffix[letter] if base.suffix else self.trie_root
            base = base.suffix
        return potential

    def _put_suffixes(self, node, letter):
        cursor = node.parent
        while cursor:
            suffix = cursor[letter]
            if suffix and suffix != node:
                node.suffix = suffix
                break
            elif cursor == self.trie_root:
                node.suffix = self.trie_root
                break
            cursor = cursor.suffix
        for key, child in node.children.items():
            self._put_suffixes(child, key)
    
    def _put_dict_suffixes(self, node):
        cursor = node.suffix
        while cursor:
            if cursor.in_dict:
                node.dict_suffix = cursor
            cursor = cursor.suffix
        for child in node.children.values():
            self._put_dict_suffixes(child)

    def __repr__(self):
       return self.trie_root.__repr__()

trie = Trie(['a', 'ab', 'bab', 'bc', 'bca', 'c', 'caa'], [1, 2, 3, 4, 5, 6])
trie.count('abccab')

7

In [41]:
with open('input.txt') as inputs:
    n = int(inputs.readline())
    genes = inputs.readline().rstrip().split()
    health = list(map(int, inputs.readline().rstrip().split()))
    s = int(inputs.readline())

    least = float('inf')
    most = float('-inf')

    for s_itr in range(s):
        firstLastd = inputs.readline().split()

        first = int(firstLastd[0])
        last = int(firstLastd[1])
        dna = firstLastd[2]
        
        score = dna_health(health, genes, first, last, dna)

        if score < least:
          least = score
        if score > most:
          most = score
    
    print(least, most)

NameError: name 'dna_health' is not defined